# Activation Function (활성화 함수)
> 인공신경망에서 사용되는 은닉층을 활성화하기 위한 함수이다. `입력을 정규화(Normalization)하는 과정`

- 활성화: 인공신경망의 뉴런의 출력 값을 선형에서 비선형으로 변환

## 이진 분류
> 이진 분류란 규칙에 따라 입력된 값을 두 그룹으로 분류하는 작업을 의미, True & False 형태로 결과를 분류하기 때문에 논리 회귀(Logistic Regression) 또는 논리 분류 (Logistic Classification)라고도 부른다.

### 시그모이드 함수(Sigmoid Function)
> S자형 곡석 모양으로, 반환값은 0~1 또는 -1~1의 범위를 갖는다.
- 시그모이드 함수의 x의 계수에 따라 S자형 곡선이 완만한 경사를 갖게 될지, 급격한 경사를 갖게 될지 설정할수 있다.
- 시그모이드 함수의 계수가 0에 가까워질수록 완만한 경사를 갖게 되며, 0에서 멀어질수록 급격한 경사를 갖게 된다.
- 출력값 범위가 0~1사이로 제한됨으로써 정규화 중 기울기 폭주(Exploding Gradient)문제가 발생하지 않고 미분 식이 단순한 형태를 지닌다.
- 시그모이드 함수는 기울기 폭주를 방지하는 대신 기울기 소실(Vanishing Gradient) 문제를 일으킨다. 신경망은 기울기를 이용해 최적화된 값을 찾아가는데, 계층이 많아지면 점점 값이 0에 수렴되는 문제가 발생해 성능이 떨어진다.

### 이진 교차 엔트로피
> 평균 제곱 오차는 예측값과 실제값의 차이가 작으면 계산되는 오차 또한 크기가 작아져 학습을 원활하게 진행하기 어렵다. 이러한 경우를 방지하고자, 이진 교차 엔트로피(Binary Cross Entropy, BCE)를 오차 함수로 사용한다.

- 이진 교차 엔트로피는 로그 함수를 활용해 오차 함수를 구현한다.

## Pytorch 활용 이진 분류


In [4]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:, 1].values
    self.x3 = df.iloc[:, 2].values
    self.y = df.iloc[:, 3].values
    self.length = len(df)
    
  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y
  
  def __len__(self):
    return self.length
      
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(
      nn.Linear(3, 1),
      nn.Sigmoid()
    )
    
  def forward(self, x):
    x = self.layer(x)
    return x
  
dataset = CustomDataset('../dataset/binary.csv')
dataset_size = len(dataset)

train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

device = "mps" if torch.backends.mps.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10000):
  cost = 0.0
  
  for x,y in train_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    output = model(x)
    loss =criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    cost += loss
  cost = cost / len(train_dataloader)
  
  if(epoch + 1) % 1000 == 0:
    print(f'Epoch: {epoch + 1}, Cost: {cost}')
    
with torch.no_grad():
  model.eval()
  for x, y in validation_dataloader:
    x = x.to(device)
    y = y.to(device)
    
    outputs = model(x)
    
    print(outputs)
    print(outputs >= torch.FloatTensor([0.5]).to(device))
    print("--------------------")

/var/folders/2y/2cp_6ft96fn709961nxvzc8h0000gn/T/ipykernel_12357/1022325354.py:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch: 1000, Cost: 0.6283093690872192
Epoch: 2000, Cost: 0.6162346601486206
Epoch: 3000, Cost: 0.6036415696144104
Epoch: 4000, Cost: 0.5948318839073181
Epoch: 5000, Cost: 0.5853080749511719
Epoch: 6000, Cost: 0.5780490040779114
Epoch: 7000, Cost: 0.5673131942749023
Epoch: 8000, Cost: 0.5603509545326233
Epoch: 9000, Cost: 0.5488082766532898
Epoch: 10000, Cost: 0.5427228808403015
tensor([[0.5064],
        [0.6308],
        [0.5246],
        [0.6462]], device='mps:0')
tensor([[True],
        [True],
        [True],
        [True]], device='mps:0')
--------------------
tensor([[0.4346],
        [0.7428],
        [0.4392],
        [0.5046]], device='mps:0')
tensor([[False],
        [ True],
        [False],
        [ True]], device='mps:0')
--------------------
tensor([[0.5015],
        [0.6643],
        [0.3836],
        [0.6819]], device='mps:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='mps:0')
--------------------
tensor([[0.5710],
        [0.5683],
  

## 비선형 활성화 함수(Non-linear Activations Function)
> 네트워크에 비선형성을 적용하기 위해 인공 신경망에서 사용되는 함수

- `계단 함수(Step Function)` 이진 활성화 함수라고도 하며, 퍼셉트론에서 최초 사용한 활성화 함수이다. 입력값의 합이 임계값을 넘으면 0, 못 넘으면 1을 출력한다.

- `임계값 함수(Threshold Function)` 임계값보다 크면 입력값을 그대로 전달, 임계값 보다 작으면 특정값으로 변환한다. 입력에 대한 함수의 기울기를 계산할 수 없으므로 네트워크를 최적화하기 어려워 사용되지 않는 함수이다.

- `시그모이드 함수(Sigmoid Function)` 모든 입력값을 0과 1 사이의 값으로 매핑하다. 시그모이드 함수는 은닉층에서는 활성화 함수 사용하지 않으며, 주로 출력층에서만 사용한다.

- `하이퍼볼릭 탄젠트 함수(Hyperbolic Tangent Function)` 출력값의 중심이 0이며, 출력값이 -1~1의 범위를 가져 시그모이드 함수에서 발생하지 않는 음수 값을 반환할 수 있다.

- `ReLU 함수(Rectified Linear Unit Function)` 0보다 작거나 같으면 0을 반환, 0보다 크면 선형 함수에 값을 대입하는 구조를 가진다. ReLU함수는 선형함수에 대입하므로 입력값이 양수 라면 출력값이 제한되지 않아 기울기 소실이 발생하지 않는다. 순전파나 역전파 과정이 매우 빠르지만 입력값이 음수인 경우 항상 0을 반환하므로 가중치나 편향이 갱신되지 않을 수 있다. 가중치의 합이 음수가 되면, 해당 노드는 더 이상 값을 갱신하지 않아 죽은 뉴런(Dead Neuron, Dying ReLU)이 된다.

- `LeakyReLU 함수` 음수 기울기를 제어하여, 죽은 뉴런 현상을 방지하기 위해 사용된다. 음수인 경우 작은 값이라도 출력시켜 기울기를 갱신하게 한다.

- `PReLU 함수(Parametric Rectified Linear Unit Function)` 음수 기울기 값을 고정값이 아닌 학습을 통해 갱신되는 값으로 간주한다. 즉, 음수 기울기는 지속해서 값이 변경된다.

- `ELU 함수(Exponential Linear Unit Function)` 지수 함수를 사용하여 부드러운 곡선 형태를 갖는다. 경사 하강법의 수렴속도가 비교적 빠르지만, 학습 속도는 더 느려진다. 해당 함수는 데이터의 복잡한 패턴과 관계를 학습하는 네트워크의 능력을 향상시키는 데 도움이 될 수 있다.

- `소프트맥스 함수(Softmax Function)` 차원 벡터에서 특정 출력값이 k번째 클래스에 속할 확률을 계산한다. 은닉층에서 사용하지 않고 출력층에서 사용된다.

# 순전파와 역전파

- `순전파(Forward Propagation)` 순방향 전달이라고도 하며 입력이 주어지면 신경망의 출력을 계산하는 프로세스이다.

- `역전파(Back Propagation)` 순전파 방향과 반대로 연산이 진행된다. 학습과정에서 네트워크의 가중치와 편향은 예측된 출력값과 실제 출력값 사이의 오류를 최소화하기 위해 조정된다. 그러므로 순전파 과정을 통해 나온 오차를 활용해 각 계측의 가중치와 편향을 최적화 한다.